#Non convolutional GAN for generating MNIST images


##introduction
Hi everyone. In this notebook, we are going to make a GAN using dense layer which is going to be trained on the MNIST dataset.

##Data preprocessing
In here we import essential libraries and preprocess our dataset

In [8]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.backend import clear_session
from tensorflow.keras.utils import disable_interactive_logging

from tensorflow.keras.datasets import mnist

In [2]:
#load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("x shape:", x_train.shape)

x shape: (60000, 28, 28)


In [3]:
x_train = (x_train.astype(np.float32) - 127.5)/127.5 #normalize pixel values (from -1 to +1) - this is better for relu function
x_train = x_train.reshape(60000, 784) #turn our 2D image into a 1D list (28x28 = 784)

##AI model
In here we create our model which consists of two models itself

###Generator
The generator model gets a random generated image (noisy image) and generates images similar to to the images that we want

###Discriminator
The discriminator tells us how much does the generated images resemble the images in the training dataset. We use this model to tell the generator how much error does it's generated image have.<br /><br />
In short, the generator should do it's best to get the approval of the discriminator.

In [4]:
#The values in this code were suggested in a paper published on arxiv

#the results are a little better when the dimensionality of the random vector is only 10.
#the dimensionality has been left at 100 for consistency with other GAN implementations.
randomDim = 100

#optimizer
adam = Adam(learning_rate=0.0002, beta_1=0.5)

#generator
generator = Sequential()
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer='adam')

#discriminator
discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=784, kernel_initializer=RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

#combined network
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Let's add some utility functions

In [5]:
#plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(lossHistory[0], label='Generative loss')
    plt.plot(lossHistory[1], label='Discriminitive loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/gan_loss_epoch_%d.png' % epoch)

#create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/gan_generated_image_epoch_%d.png' % epoch)

#save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    #we save our model in .keras format because the original .h5 has been deprecated
    generator.save('models/gan_generator_epoch_%d.keras' % epoch)
    discriminator.save('models/gan_discriminator_epoch_%d.keras' % epoch)

And this is the train function<br />
It trains both models, generates images and saves the model in each epoch

In [6]:
def train(epochs=1, batchSize=128):
    #the loss history of both the generator and the discriminator is saved in this list
    #the generator's loss history at index 0
    #and the discriminator's at index 1
    lossHistory = [[], []]

    batchCount = int(x_train.shape[0] / batchSize)
    print('Epochs:', epochs)
    print('Batch size:', batchSize)
    print('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in range(batchCount):
            #get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = x_train[np.random.randint(0, x_train.shape[0], size=batchSize)]

            #generate fake MNIST images
            generatedImages = generator.predict(noise)
            # print np.shape(imageBatch), np.shape(generatedImages)
            X = np.concatenate([imageBatch, generatedImages])

            #labels for generated and real data
            yDis = np.zeros(2*batchSize)
            #one-sided label smoothing
            yDis[:batchSize] = 0.9

            #train discriminator
            discriminator.trainable = True
            dLoss = discriminator.train_on_batch(X, yDis)

            #train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gLoss = gan.train_on_batch(noise, yGen)

        #store loss of most recent batch from this epoch
        lossHistory[0].append(gLoss)
        lossHistory[1].append(dLoss)

        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)

Now let's train the model

In [ ]:
clear_session()
disable_interactive_logging() #the progress bar dosn't do us any good in this project
train(200, 256)

Epochs: 200
Batch size: 256
Batches per epoch: 234
--------------- Epoch 1 ---------------
--------------- Epoch 2 ---------------
--------------- Epoch 3 ---------------
